In [292]:
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
DATA_PATH = Path() / "../data"
#DATA_PATH = Path('/content/gdrive/My Drive/Colab Notebooks/VC_data_Cleaning_With_LLM/data')
DATA_PATH.mkdir(parents=True,exist_ok=True)

def load_data(filename, data_path=DATA_PATH,encoding='ISO-8859-1'):
    csv_path = data_path / filename
    return pd.read_csv(csv_path,encoding=encoding)

def save_data(data, filename, data_path=DATA_PATH,encoding='ISO-8859-1'):
    csv_path = data_path / filename
    data.to_csv(csv_path, index=False,encoding='ISO-8859-1')

PLOT_PATH = Path() / "../plot"
#PLOT_PATH = Path('/content/gdrive/My Drive/Colab Notebooks/VC_data_Cleaning_With_LLM/plot')
PLOT_PATH.mkdir(parents=True,exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300, transparent=True):
    path = PLOT_PATH / f"{fig_id}.{fig_extension}"
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution, transparent=transparent)

In [293]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from kor.extraction import create_extraction_chain
from kor.nodes import Object,Text, Number

In [294]:
import json
def json_dump(json_object):
    json_formatted_str = json.dumps(json_object, indent=2,ensure_ascii= False)
    print(json_formatted_str)

In [295]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [296]:
# Import the openai module
import openai

In [297]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    role: Optional[str] = Field(
        ..., description="The role or title of this person. "
                                )
    company: Optional[str] = Field(
        ..., description="The company this person is working in."
    )
    ai_related: Optional[bool] = Field(
        ..., description="Whether this person is related to AI."
    )

In [298]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract"
            "return null for the attribute's value."

            "The list we input is a list of people's biography. Each element in the list is a person's biography text."
            "You should treat every element in the list as a separate entity."

            "For each person, you should consider:"
            "If this person has several roles, we should only annotate one."
            "If this person is a AI company founder or co-founder, annotate him as a co-founder."
            "If this person is a researcher in AI feild, annotate this person as a researcher."
            "If you can't find any role, try to conclude the role from the context, if you still can't find any role, please annotate the role as NA."

            "when you extract company name, please extract the full name of the company and exclude any other punctuation mark such as @."
            "If there is no company name, please annotate the company name as NA."

            "If this person is related to AI, or anything about AI, please annotate the AI_related as true, otherwise annotate it as false."

        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [299]:
from typing import List, Optional
class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

In [300]:
import uuid
from typing import Dict, List, TypedDict
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)

class Example(TypedDict):
    """A representation of an example consisting of text input and expected tool calls.

    For extraction, the tool calls are represented as instances of pydantic model.
    """

    input: str  # This is the example text
    tool_calls: List[BaseModel]  # Instances of pydantic model that should be extracted


In [301]:
def tool_example_to_messages(example: Example) -> List[BaseMessage]:
    """Convert an example into a list of messages that can be fed into an LLM.

    This code is an adapter that converts our example to a list of messages
    that can be fed into a chat model.

    The list of messages per example corresponds to:

    1) HumanMessage: contains the content from which content should be extracted.
    2) AIMessage: contains the extracted information from the model
    3) ToolMessage: contains confirmation to the model that the model requested a tool correctly.

    The ToolMessage is required because some of the chat models are hyper-optimized for agents
    rather than for an extraction use case.
    """
    messages: List[BaseMessage] = [HumanMessage(content=example["input"])]
    openai_tool_calls = []
    for tool_call in example["tool_calls"]:
        openai_tool_calls.append(
            {
                "id": str(uuid.uuid4()),
                "type": "function",
                "function": {
                    # The name of the function right now corresponds
                    # to the name of the pydantic model
                    # This is implicit in the API right now,
                    # and will be improved over time.
                    "name": tool_call.__class__.__name__,
                    "arguments": tool_call.json(),
                },
            }
        )
    messages.append(
        AIMessage(content="", additional_kwargs={"tool_calls": openai_tool_calls})
    )
    tool_outputs = example.get("tool_outputs") or [
        "You have correctly called this tool."
    ] * len(openai_tool_calls)
    for output, tool_call in zip(tool_outputs, openai_tool_calls):
        messages.append(ToolMessage(content=output, tool_call_id=tool_call["id"]))
    return messages

In [302]:
examples = [
    (
        "Research Scientist at Google DeepMind. I tweet about deep learning (research + software), music, generative models (personal account).",
        Person(role="Research Scientist", company="Google DeepMind", interest="deep learning, music, and generative models", ai_related=True),
    ),
    (
        "I make videos.\nSkill &gt; Destiny.\nvi / vim",
        Person(role=None, company=None, interest=None, ai_related=False),
    ),
    (
        "ML Researcher, co-leading Superalignment @OpenAI. Optimizing for a post-AGI future where humanity flourishes.",
        Person(role="ML Researcher", company="OpenAI", interest="AGI", ai_related=True),
    )
]


messages = []

for text, tool_call in examples:
    messages.extend(
        tool_example_to_messages({"input": text, "tool_calls": [tool_call]})
    )

### Create an extractor

In [303]:
llm = ChatOpenAI(
    model_name = 'gpt-4-turbo-preview',
    temperature = 0,
    openai_api_key = OPENAI_API_KEY
)

In [304]:
runnable = prompt | llm.with_structured_output(
    schema=Data,
    method="function_calling",
    include_raw=False,
)

In [305]:
df = load_data('twitter_combined.csv')
# Drop NaN values from the 'bio' column
df = df.dropna(subset=['bio'])

# Combine all text in the 'bio' column into a list, considering only the first 100 texts
documents = df['bio'].head(20).tolist()

In [306]:
documents

['Making medicines differently to bring better drugs faster to the patients who benefit most with ML and data at scale.',
 '#LearnWithoutLimits on Coursera. Access online courses and degrees from world-class universities and companies. Visit https://t.co/iZWWGG1ypn for support.',
 'The leading news source for higher education. Get our newsletters: https://t.co/zfAnZfGsHc',
 "Sharing things I'm learning through my foundation work and other interests.",
 'Policy research at @openai. I mostly tweet about AI, animals, and sci-fi. He/him. Views my own.',
 'Co-Founder of OpenAI',
 'Chief Scientist, Google DeepMind and Google Research. Co-designer/implementor of things like @TensorFlow, MapReduce, Bigtable, Spanner, Gemini .. (he/him)',
 'VP of Research & Deep Learning Lead, Google DeepMind. Gemini co-lead.\n\nPast: AlphaStar, AlphaFold, AlphaCode, WaveNet, seq2seq, distillation, TF.',
 '@AnthropicAI, ONEAI OECD, co-chair @indexingai, writer @ https://t.co/3vmtHYkaTu Past: @openai, @business 

In [307]:
text='''
  Policy research at @openai. 
  I mostly tweet about AI, animals, and sci-fi. He/him. Views my own.
  Chief Scientist, Google DeepMind and Google Research. Co-designer/implementor of things like @TensorFlow, MapReduce, Bigtable, Spanner, Gemini .. (he/him)
'''
texts = ['Making medicines differently to bring better drugs faster to the patients who benefit most with ML and data at scale.',
 '#LearnWithoutLimits on Coursera. Access online courses and degrees from world-class universities and companies. Visit https://t.co/iZWWGG1ypn for support.',
 'The leading news source for higher education. Get our newsletters: https://t.co/zfAnZfGsHc']

text = documents

output = runnable.invoke({"text": text, "examples": messages})

In [308]:
output

Data(people=[Person(role='NA', company='NA', ai_related=True), Person(role='NA', company='Coursera', ai_related=False), Person(role='NA', company='NA', ai_related=False), Person(role='NA', company='NA', ai_related=False), Person(role='researcher', company='OpenAI', ai_related=True), Person(role='co-founder', company='OpenAI', ai_related=True), Person(role='Chief Scientist', company='Google DeepMind', ai_related=True), Person(role='VP of Research & Deep Learning Lead', company='Google DeepMind', ai_related=True), Person(role='NA', company='AnthropicAI', ai_related=True), Person(role='Research Scientist', company='Google DeepMind', ai_related=True), Person(role='NA', company='OpenMined', ai_related=True), Person(role='NA', company='New York University', ai_related=False), Person(role='NA', company='Google DeepMind', ai_related=True), Person(role='NA', company='NA', ai_related=False), Person(role='Professor', company='Carnegie Mellon University', ai_related=True), Person(role='Director', 

In [309]:
# Extract information to a list of dictionaries
people_data = [{"role": person.role, "company": person.company, "ai_related": person.ai_related} for person in output.people]

# Convert to DataFrame
bio_df = pd.DataFrame(people_data)

bio_df

,role,company,ai_related
0,NA,NA,True
1,NA,Coursera,False
2,NA,NA,False
3,NA,NA,False
4,researcher,OpenAI,True
5,co-founder,OpenAI,True
6,Chief Scientist,Google DeepMind,True
7,VP of Research & Deep Learning Lead,Google DeepMind,True
8,NA,AnthropicAI,True
9,Research Scientist,Google DeepMind,True
